In [6]:
### forest-disturbance-stack-v3

### This script uses several functions to create custom annual disturbance stacks. First,
### a template raster is created using a reference raster. Next, a forest mask is resampled and projected 
### to match template. Next, input disturbance rasters are subsetted to years of interest
### and resampled to match template. Finally, each input disturbance raster is stacked together,
### annually, and forest mask is applied to each stack/band.

### Returns and saves final GeoTIFF (.tif) for each year.


# 0. Setup
# 1. Template raster setup
# 2. Input rasters and metadata
# 3. Resample & Reproject Forest Mask to Template
# 4. Subset & resample input rasters to template
# 5. Create annual stacks



# ============================================================
# 0. Setup
# ============================================================

# Working directory
setwd("/home/jovyan/data-store/forest-disturbance-stack-v3")
here::i_am("README.md")

# Packages
packages <- c("here", "terra", "fs")
installed <- packages %in% installed.packages()[, "Package"]
if (any(!installed)) install.packages(packages[!installed])

library(here)
library(terra)
library(fs)

# Create new data directories
dir.create(here("data", "derived", "not_resampled"), showWarnings = FALSE, recursive = TRUE)
dir.create(here("data", "derived", "resampled"), showWarnings = FALSE, recursive = TRUE)

# Make sure input rasters and forest mask are placed in data/derived/not_resampled

# Dedicated temp directory for CyVerse (no local SSD)
cyverse_tmp <- here("tmp_terra")
dir.create(cyverse_tmp, showWarnings = FALSE, recursive = TRUE)

terraOptions(
  tempdir = cyverse_tmp,
  memfrac = 0.4,    # use 40% of RAM
  memmax = 180000,   # cap memory at 64 GB
  progress = 1,
  threads = 1       # avoid I/O contention
)
Sys.setenv(TMPDIR = cyverse_tmp)

here() starts at /home/jovyan/data-store/forest-disturbance-stack-v3



In [2]:
# ============================================================
# 1. Template raster setup
# ============================================================

# Template path
template_path <- here("data", "derived", "resampled", "template_30m_singleband.tif")

if (!file.exists(template_path)) {
  
  message("Creating template raster ...")
  
  # Reference raster (any input raster is fine)
  ref_raster_path <- here("data", "derived", "not_resampled", "wildfire_id.tif")
  ref <- rast(ref_raster_path)
  
  # Build a single-band template for alignment
  single_template <- rast(
    ext = ext(ref),
    res = res(ref),
    crs = crs(ref),
    nlyr = 1
  )
  values(single_template) <- NA_real_
  
  # Ensure directory exists
  dir.create(dirname(template_path), showWarnings = FALSE, recursive = TRUE)
  
  # Save template
  writeRaster(
    single_template,
    template_path,
    datatype = "FLT4S",
    overwrite = TRUE,
    gdal = c("COMPRESS=LZW", "TILED=YES", "BIGTIFF=YES")
  )
  
  message("✅ Template created at: ", template_path)
  
} else {
  message("✅ Template already exists at: ", template_path, " – skipping creation")
}

Creating template raster ...



✅ Template created at: /home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/template_30m_singleband.tif



In [3]:
# ============================================================
# 2. Input rasters and metadata
# ============================================================

raster_paths <- c(
  wildfire = here("data", "derived", "not_resampled", "wildfire_id.tif"),
  biotic   = here("data", "derived", "not_resampled", "biotic_gridded_1km_all_years_severity.tif"),
  pdsi     = here("data", "derived", "not_resampled", "pdsi_annual.tif"),
  hd       = here("data", "derived", "not_resampled", "hd_fingerprint.tif")
)

# Starting years of each raster (first band)
start_years <- c(wildfire = 1984, biotic = 1997, pdsi = 2000, hd = 1958)

# Forest mask
forest_mask_path <- here("data", "derived", "not_resampled", "relaxed_forest_mask_2000_2020.tif")
forest_mask_resampled <- here("data", "derived", "resampled", "forest_mask_30m_resampled.tif")

# Years of interest
years_of_interest <- 2000:2020

In [4]:
# ============================================================
# 3. Resample & Reproject Forest Mask to Template
# ============================================================

forest_mask_resampled <- here("data", "derived", "resampled", "forest_mask_30m_resampled.tif")
template_path <- here("data", "derived", "resampled", "template_30m_singleband.tif")

if (!file.exists(forest_mask_resampled)) {
  
  message("Resampling and reprojecting forest mask ...")
  
  # Load original forest mask
  mask_r <- rast(forest_mask_path)
  if (nlyr(mask_r) > 1) mask_r <- mask_r[[1]]  # single band
  
  # Load template as SpatRaster
  template_r <- rast(template_path)
  
  # Reproject + resample to match template CRS, extent, resolution
  mask_aligned <- project(mask_r, template_r, method = "near")  # nearest neighbor for categorical
  
  # Binarize: 1 = forest, 0 = non-forest
  mask_aligned <- mask_aligned > 0
  
  # Write raster safely
  writeRaster(
    mask_aligned,
    forest_mask_resampled,
    overwrite = TRUE,
    datatype = "INT1U",
    gdal = c("COMPRESS=LZW", "BIGTIFF=YES", "TILED=YES")
  )
  
  # Quick check
  mask_check <- rast(forest_mask_resampled)
  message("✅ Forest mask ready: ", forest_mask_resampled)
  message("   CRS: ", crs(mask_check))
  message("   Extent: ", ext(mask_check))
  message("   Resolution: ", res(mask_check))
  
} else {
  message("✅ Forest mask already exists at: ", forest_mask_resampled, " – skipping resampling")
}

Resampling and reprojecting forest mask ...



✅ Forest mask ready: /home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/forest_mask_30m_resampled.tif

   CRS: PROJCRS["NAD83 / Conus Albers",
    BASEGEOGCRS["NAD83",
        DATUM["North American Datum 1983",
            ELLIPSOID["GRS 1980",6378137,298.257222101,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4269]],
    CONVERSION["Conus Albers",
        METHOD["Albers Equal Area",
            ID["EPSG",9822]],
        PARAMETER["Latitude of false origin",23,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8821]],
        PARAMETER["Longitude of false origin",-96,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8822]],
        PARAMETER["Latitude of 1st standard parallel",29.5,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8823]],
        PARAMETER["Latitude of 2nd standard parallel",45

In [5]:
# ============================================================
# 4. Subset & resample input rasters to template
# ============================================================

# Helper functions
compute_band_indices <- function(start_year, n_bands, years_of_interest) {
  # Create vector of all years in the raster
  all_years <- seq(from = start_year, length.out = n_bands)
  
  # Return the indices of bands that match the years of interest
  which(all_years %in% years_of_interest)
}

default_resample_method <- function(file) {
  # Categorical rasters use "near", continuous rasters use "bilinear"
  if (grepl("wildfire|biotic|hd", file, ignore.case = TRUE)) return("near")
  return("bilinear")
}

# Subset and resample function
subset_then_resample <- function(template_path,
                                 raster_paths,
                                 start_years,
                                 years_of_interest,
                                 tmp_dir = "tmp_terra",
                                 n_threads = 8,
                                 overwrite = FALSE) {
  
  if (!dir.exists(tmp_dir)) dir.create(tmp_dir, recursive = TRUE)
  
  # Load template for extent/resolution
  tmpl <- rast(template_path)
  res_x <- res(tmpl)[1]
  res_y <- res(tmpl)[2]
  ex <- ext(tmpl)
  
  resampled_outs <- character(length(raster_paths))
  names(resampled_outs) <- basename(raster_paths)
  
  # Track which rasters were skipped
  skipped_subset <- character()
  skipped_resample <- character()
  
  # Iterate over rasters
  for (i in seq_along(raster_paths)) {
    in_file <- raster_paths[[i]]
    start_year <- start_years[[i]]
    
    src_rast <- rast(in_file)
    n_bands <- nlyr(src_rast)
    rm(src_rast); gc()
    
    # Compute band indices for desired years
    bands_needed <- compute_band_indices(start_year, n_bands, years_of_interest)
    
    if (length(bands_needed) == 0) {
      message("⚠️ No overlapping years in ", basename(in_file))
      next
    }
    
    base <- tools::file_path_sans_ext(basename(in_file))
    subset_file <- file.path(tmp_dir, paste0(base, "_", min(years_of_interest), "_", max(years_of_interest), "_subset.tif"))
    resampled_file <- file.path(tmp_dir, paste0(base, "_30m_resampled.tif"))
    
    # Create subset (only needed bands)
    if (!file.exists(subset_file) || overwrite) {
      b_args <- paste(sprintf("-b %d", bands_needed), collapse = " ")
      cmd_tr <- sprintf("gdal_translate %s %s %s",
                        b_args, shQuote(in_file), shQuote(subset_file))
      message("🪶 Subsetting bands for ", basename(in_file))
      ret <- system(cmd_tr)
      if (ret != 0) stop("gdal_translate failed for: ", in_file)
    } else {
      message("✅ Subset already exists for ", basename(in_file), " – skipping subsetting")
      skipped_subset <- c(skipped_subset, basename(in_file))
    }
    
    # Resample to 30m template
    if (!file.exists(resampled_file) || overwrite) {
      method <- default_resample_method(in_file)
      cmd_warp <- sprintf(
        paste(
          "gdalwarp -tr %s %s -r %s",
          "-te %s %s %s %s",
          "-multi -wo NUM_THREADS=%d",
          "-co COMPRESS=DEFLATE -co BIGTIFF=YES",
          "%s %s"
        ),
        res_x, res_y, method,
        ex[1], ex[3], ex[2], ex[4],
        n_threads, shQuote(subset_file), shQuote(resampled_file)
      )
      message("🌀 Resampling ", basename(in_file), " to 30m grid")
      ret2 <- system(cmd_warp)
      if (ret2 != 0) stop("gdalwarp failed for: ", subset_file)
    } else {
      message("✅ Resampled file already exists for ", basename(in_file), " – skipping resampling")
      skipped_resample <- c(skipped_resample, basename(in_file))
    }
    
    resampled_outs[i] <- normalizePath(resampled_file)
  }
  
  # Summary message
  if (length(skipped_subset) > 0) {
    message("📄 Subset skipped for: ", paste(skipped_subset, collapse = ", "))
  }
  if (length(skipped_resample) > 0) {
    message("📄 Resampling skipped for: ", paste(skipped_resample, collapse = ", "))
  }
  
  message("🎉 All resampled files are ready in: ", normalizePath(tmp_dir))
  
  return(resampled_outs)
}

# Run resampling
resampled_files <- subset_then_resample(
  template_path = template_path,
  raster_paths = raster_paths,
  start_years = start_years,
  years_of_interest = years_of_interest,
  overwrite = FALSE
)


🪶 Subsetting bands for wildfire_id.tif

🌀 Resampling wildfire_id.tif to 30m grid

🪶 Subsetting bands for biotic_gridded_1km_all_years_severity.tif

🌀 Resampling biotic_gridded_1km_all_years_severity.tif to 30m grid

🪶 Subsetting bands for pdsi_annual.tif

🌀 Resampling pdsi_annual.tif to 30m grid

🪶 Subsetting bands for hd_fingerprint.tif

🌀 Resampling hd_fingerprint.tif to 30m grid

🎉 All resampled files are ready in: /home/jovyan/data-store/forest-disturbance-stack-v3/tmp_terra



In [7]:
# ============================================================
# 5. Move temp files to /resampled folder for GDAL command line next step
# ============================================================

# Source and destination folders
src_dir <- here("tmp_terra")
dst_dir <- here("data", "derived", "resampled")

# Make sure destination exists
dir_create(dst_dir)

# List all GeoTIFFs (.tif) in temp_terra
tifs <- dir_ls(src_dir, glob = "*.tif")

# Preview before moving (safety check)
print(tifs)

# Move files (use file_move to rename in place; or file_copy if you want to duplicate)
file_copy(tifs, dst_dir, overwrite = FALSE)

# Verify
dir_ls(dst_dir, glob = "*.tif")


/home/jovyan/data-store/forest-disturbance-stack-v3/tmp_terra/biotic_gridded_1km_all_years_severity_2000_2020_subset.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/tmp_terra/biotic_gridded_1km_all_years_severity_30m_resampled.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/tmp_terra/hd_fingerprint_2000_2020_subset.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/tmp_terra/hd_fingerprint_30m_resampled.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/tmp_terra/pdsi_annual_2000_2020_subset.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/tmp_terra/pdsi_annual_30m_resampled.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/tmp_terra/wildfire_id_2000_2020_subset.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/tmp_terra/wildfire_id_30m_resampled.tif


/home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/biotic_gridded_1km_all_years_severity_2000_2020_subset.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/biotic_gridded_1km_all_years_severity_30m_resampled.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/forest_mask_30m_resampled.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/hd_fingerprint_2000_2020_subset.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/hd_fingerprint_30m_resampled.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/pdsi_annual_2000_2020_subset.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/pdsi_annual_30m_resampled.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/template_30m_singleband.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/wildfire_id_2000_2020_subset.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/wildfire_id_30m_resampled.tif

In [ ]:
### Below is depricated. Switching to command line GDAL instead




# ============================================================
# 5. Create annual stacks
# ============================================================

create_annual_stacks <- function(resampled_paths,
                                 forest_mask_resampled,
                                 years_of_interest,
                                 out_dir = here("data", "derived", "annual_stacks"),
                                 out_prefix = "disturbance_stack",
                                 overwrite = TRUE) {

  if (!dir.exists(out_dir)) dir.create(out_dir, recursive = TRUE)

  # Load forest mask
  mask_r <- rast(forest_mask_resampled)
  mask_vals <- mask_r > 0

  n_inputs <- length(resampled_paths)
  input_names <- names(resampled_paths)

  # Open all resampled rasters once
  rasters <- lapply(resampled_paths, rast)

  for (i in seq_along(years_of_interest)) {
    yr <- years_of_interest[i]
    out_file <- file.path(out_dir, sprintf("%s_%d.tif", out_prefix, yr))
    if (file.exists(out_file) && !overwrite) next

    message("===============================================")
    message("⏳ Processing year: ", yr)
    message("Output: ", out_file)

    # Prepare output raster with 4 bands
    out_r <- rast(mask_r, nlyr = n_inputs)
    out <- writeStart(out_r, filename = out_file,
                      overwrite = TRUE, datatype = "FLT4S",
                      gdal = c("COMPRESS=LZW", "BIGTIFF=YES", "TILED=YES"))

    bs <- blockSize(mask_r)
    message("Number of blocks: ", bs$n)

    for (b in 1:bs$n) {
      rows <- bs$row[b]
      nrows <- bs$nrows[b]

      m_vals <- getValues(mask_vals, row = rows, nrows = nrows)
      out_block <- matrix(NA_real_, nrow = length(m_vals), ncol = n_inputs)

      for (j in seq_along(rasters)) {
        r <- rasters[[j]]
        band_index <- which((start_years[j] + 0:(nlyr(r)-1)) == yr)

        if (length(band_index) == 0) {
          out_block[, j] <- NA_real_
        } else {
          vals <- getValues(r[[band_index]], row = rows, nrows = nrows)
          vals[m_vals == 0] <- 0
          out_block[, j] <- vals
        }

        # Progress per input raster
        message(sprintf("    ✅ %s band for year %d processed (block %d/%d)",
                        input_names[j], yr, b, bs$n))
      }

      writeValues(out, out_block, rows)
      rm(out_block, m_vals); gc()
    }

    out <- writeStop(out)
    names(out) <- input_names
    message("✅ Written annual stack for year ", yr)
    message("===============================================")
    gc()
  }

  message("🎉 All annual stacks created at: ", normalizePath(out_dir))
}

# Run annual stack creation
create_annual_stacks(
  resampled_paths = resampled_files,
  forest_mask_resampled = forest_mask_resampled,
  years_of_interest = years_of_interest,
  out_prefix = "disturbance_stack",
  overwrite = TRUE
)